In [2]:
import numpy as np
import rasterio
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask

ModuleNotFoundError: No module named 'rasterio'

In [19]:
# Modify Raster Values Based on the Vector Mask
# Read the vector mask to get the modification percentages
mask_gdf = gpd.read_file("vector_mask_milan.gpkg")

# Open the raster to modify
with rasterio.open("asphalt.tif") as src:
    # Read the entire raster data into memory
    raster_data = src.read(1)  # Read the first band (assuming single-band raster)
    transform = src.transform
    crs = src.crs
    
    # Loop over each polygon in the vector mask
    for idx, row in mask_gdf.iterrows():
        # Extract the polygon geometry and modification percentage
        geo = [row["geometry"]]  # Convert geometry to list
        modification_percentage = row["asphalt"]

        # Create a mask based on the current polygon
        out_image, out_transform = mask(src, geo, crop=False)  # Don't crop, just mask
        out_image = out_image[0]  # For single-band raster, use the first band
        
        # Create a mask where the polygon is and modify those pixels
        modified_image = out_image + (out_image * (modification_percentage / 100.0))
        modified_image = np.clip(modified_image, 0, 100)  # Ensure values stay between 0 and 100

        # Find the pixel coordinates for the polygon in the original raster
        polygon_mask, _ = mask(src, geo, crop=False)  # Mask the raster with the polygon (no crop)
        polygon_mask = polygon_mask[0]  # Convert to 1D for single-band raster

        # Apply the modification to the original raster (only where the polygon mask is True)
        raster_data[polygon_mask > 0] = modified_image[polygon_mask > 0]

# Save the modified raster
with rasterio.open(
    "modified_asphalt.tif", "w", driver="GTiff", height=raster_data.shape[0], width=raster_data.shape[1],
    count=1, dtype=raster_data.dtype, crs=crs, transform=transform
) as dst:
    dst.write(raster_data, 1)  # Write the modified data to the raster file

print("Raster modification completed and saved as modified TIFF.")

Raster modification completed and saved as modified TIFF.
